In [1]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [2]:
loader = UnstructuredPDFLoader("../Downloads/Ayurveda-Cookbook.pdf")

In [3]:
data = loader.load()

100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 1355.52it/s]


In [4]:
print (f'You have {len(data)} document(s) in your data')
print (f'There are {len(data[0].page_content)} characters in your document')

You have 1 document(s) in your data
There are 111751 characters in your document


In [5]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

In [6]:
print (f'Now you have {len(texts)} documents')

Now you have 139 documents


In [25]:
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
import pinecone

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY =os.getenv.('OPENAI_API_KEY')
PINECONE_API_KEY = os.getenv.('PINECONE_API_KEY')
PINECONE_API_ENV = os.getenv.('PINECONE_API_ENV')

: 

In [42]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [43]:
pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_API_ENV,
)
index_name = "testing"

In [44]:
docsearch = Pinecone.from_texts([t.page_content for t in texts], embeddings, index_name=index_name)


In [45]:
from langchain.llms import OpenAI
from langchain.chains.question_answering import load_qa_chain

In [46]:
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)
chain = load_qa_chain(llm, chain_type="stuff")

In [47]:
query = "What is Ayurveda?"
docs = docsearch.similarity_search(query, include_metadata=True)

In [48]:
chain.run(input_documents=docs, question=query)

' Ayurveda is an ancient wisdom of India that embraces health knowledge from more than 2000 years of experience. It is a holistic approach to health and wellness that focuses on the balance of the body, mind, and spirit. It is based on the concept of tridosha, which is the balance of three energies in the body: vata, pitta, and kapha. Ayurveda also emphasizes the importance of proper diet, lifestyle, and the use of herbs and other natural remedies to maintain health and prevent disease.'